## Анализириуем баллы за тервер и матстат (до-онлайн эра vs онлайн эра)

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sts

In [2]:
data = pd.read_html('economics_probability_2019_20 - Google Диск.html')

In [3]:
data = data[0][2:398]
data = pd.DataFrame(data)

In [4]:
data.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51
2,2.0,1,Алиев,Эльнур,Гашимович,БЭК181,NaN,NaN,egaliev_1@edu.hse.ru,0,...,5,NaN,18,68,7.6,10,NaN,10,NaN,5.90
3,3.0,2,Алтунина,Анастасия,Игоревна,БЭК181,NaN,NaN,aialtunina@edu.hse.ru,0,...,10,NaN,20,92,9.9,10,NaN,10,NaN,6.27
4,4.0,3,Байкалова,Дарья,Александровна,БЭК181,NaN,NaN,dabaykalova@edu.hse.ru,0,...,9,NaN,20,88,9.5,10,NaN,10,NaN,6.68
5,5.0,226,Антипина,Екатерина,Дмитриевна,БЭК184,NaN,NaN,edantipina@edu.hse.ru,0,...,4,NaN,12,55,6.2,10,NaN,9,NaN,4.84
6,6.0,5,Васьков,Алексей,Васильевич,БЭК181,NaN,NaN,avvaskov@edu.hse.ru,0,...,NaN,NaN,0,0,0.0,10,NaN,10,NaN,3.17


In [5]:
marks_1 = [] #kr1
for i in data['Unnamed: 10'].dropna():
    marks_1.append(float(i))
marks_1 = np.array(marks_1)
marks_1.mean(), marks_1.var()

(52.0505076142132, 835.5919921281146)

In [6]:
marks_2 = [] #kr2
for i in data['Unnamed: 11'].dropna():
    marks_2.append(float(i))
marks_2 = np.array(marks_2)
marks_2.mean(), marks_2.var()

(37.611111111111114, 654.7212401795734)

In [7]:
terver = (marks_1+marks_2[2:])/2 #среднее за кр по терверу
terver.mean(), terver.var()

(44.72829949238579, 494.33261918949216)

In [8]:
marks_3 = [] #kr3
for i in data['Unnamed: 32'].dropna():
    marks_3.append(float(i))
marks_3 = np.array(marks_3)*10
marks_3.mean(), marks_3.var()

(55.87786259541985, 688.1886447953693)

In [9]:
marks_4 = [] #kr4
for i in data['Unnamed: 46'].dropna():
    marks_4.append(float(i))
marks_4 = np.array(marks_4)*10
marks_4.mean(), marks_4.var()

(51.26208651399491, 622.900776308037)

In [10]:
matstat = (marks_3+marks_4)/2 #среднее за кр по матстату
matstat.mean(), matstat.var()

(53.56997455470738, 598.8621519077495)

### Найдем вероятность отклонения оценки по терверу на величину бОльшую, чем разница средних оценок за два предмета
Примем среднюю оценку по терверу за матожидание. Используя нормальную аппроксимацию можно вычислить данную вероятность. Предполагаем, что тервер и матстат одинакового уровня сложности. 

p(x - terver.mean() > matstat.mean() - terver.mean()) = ?

In [11]:
probability = 1 - sts.norm.cdf((matstat.mean() - terver.mean()) / terver.std())
probability

0.34543602277303

##### То есть вероятность отклонения оценки за тервер на величину отклонения среднего за матстат от среднего за тервер равна 0.34543602277303. Вероятность на порядок ниже 0.5 и, учитывая одинаковый уровень сложности предметов, можно сделать вывод, что дистанционка пошла на пользу ;)

### Построим доверительный интервал для разности матожиданий оценок за тервер и матстат

In [12]:
average_terver = terver.mean()
average_matstat = matstat.mean()
alpha = 0.95
sigma_terver =terver.var()
sigma_matstat = matstat.var()
n_x = data['Unnamed: 10'].dropna().shape[0]
n_y = data['Unnamed: 11'].dropna().shape[0]
left = average_terver - average_matstat - sts.norm.ppf((1+alpha)/2)*(sigma_terver / n_x + sigma_matstat / n_y)**0.5
right = average_terver - average_matstat + sts.norm.ppf((1+alpha)/2)*(sigma_terver / n_x + sigma_matstat/ n_y)**0.5
print('Доверительный интервал для разности матожиданий с уровнем доверия 0.95:', (left, right))

Доверительный интервал для разности матожиданий с уровнем доверия 0.95: (-12.101896780827218, -5.581453343815966)


### Проверим гипотезу о равенстве дисперсий оценок за тервер и матстат

H0: sigma_2_terver = sigma_2_matstat

H1: sigma_2_terver $\neq$ sigma_2_matstat

In [13]:
alpha = 0.05
sigma_2_terver = terver.var()
sigma_2_matstat = matstat.var()
test = sigma_2_matstat / sigma_2_terver
zona_neotverzheniya = (sts.f.ppf(alpha/2, matstat.shape[0], terver.shape[0]), sts.f.ppf(1-alpha/2, matstat.shape[0], terver.shape[0]))
if zona_neotverzheniya[1] > test > zona_neotverzheniya[0]:
    print('H0 не отвергается')
else:
    print('H0 отвергается')
print('test = ', test, 'Зона неотвержения гипотезы H0 =', zona_neotverzheniya)

H0 не отвергается
test =  1.2114558672855618 Зона неотвержения гипотезы H0 = (0.8204454136829143, 1.2188194006595552)


### Критерий Пирсона

Проверяем гипотезу о том, что вероятности получить тот или иной балл по матстату одинаковы. Уровень значимости - 0.01

In [14]:
stat = pd.Index(matstat)
frequency = []
for i in stat.value_counts():
    frequency.append(i)
frequency = np.array(frequency)
frequency #вектор количества студентов, получивших определенный балл

array([33,  9,  8,  8,  7,  7,  6,  6,  6,  6,  6,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1])

In [15]:
s = frequency.shape[0]
probabilities = np.array([1/s]*s) #вектор одинаковых вероятностей размерностью с количеством различных значений (параметр s)

In [18]:
prob = probabilities
values = frequency
n = frequency.shape[0]
znachenie_kriteria_pirsona = 0
for i in range(len(values)):
    znachenie_kriteria_pirsona += (values[i]-n*prob[i])**2 / (n*prob[i])
alpha = 0.01
k = 0
if sts.chi2.ppf(1-alpha, df = s-k-1) < znachenie_kriteria_pirsona:
    print('Гипотеза отвергается')
else:
    print('Гипотеза не отвергается')
print('Значение критерия Пирсона =', znachenie_kriteria_pirsona)
print('Критическая область: z_alpha > ',(sts.chi2.ppf(1-alpha, df = s-k-1)))

Гипотеза отвергается
Значение критерия Пирсона = 1800.0
Критическая область: z_alpha >  181.84033712682086
